<a href="https://colab.research.google.com/github/stebechoi/CP2/blob/Hwkdir/3.%20(MovieLense100K)_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MovieLense 데이터를 이용한 영화 추천 시스템(Collaborative filtering: 협업필터링)

## CF
- Collaborative Filtering 협업 필터링 </br>
  사용자와 비슷한 다른 사용자의 평가를 파악하여 이용
    - 장점: 아이템 특성에 의존하지 않는다.
    - 단점
      - 평가되지 않은 아이템은 추천하지 않는다(new item problem)
      - 보통 가장 인기 있는 아이템이 추천된다.
      - 비슷한 유저 군이 존재하는 사용자 그룹이 필요하다.(cold start problem for new users)

### 데이터 로드

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/gdrive')
# 작업 디렉토리 변경 change directory
import os
os.chdir('/content/gdrive/My Drive/Colab Notebooks/CP/CP2')

In [22]:
os.getcwd()

'/content/gdrive/MyDrive/Colab Notebooks/CP/CP2'

In [21]:
ls

'3_27_28_CP2_RecSys_Note_16th.ipynb의 사본'
'AIB_16기_K팀_01주차_공유사항[오혜진, 최영조].gdoc'
'AIB_16기_K팀_02주차_공유사항[오혜진, 최영조].gdoc'
'AIB_16기_K팀_03주차_공유사항[오혜진, 최영조].gdoc'
 CP2-eda~CB
 CP2.ipynb
 ml-100k/
'(MovieLense100K) CF .ipynb'
 movielense-100k.zip
'(MovieLense100K) 전처리 및 EDA.ipynb'
 movielense_merged.csv


In [23]:
df = pd.read_csv('movielense_merged.csv')

df_us = df[['user_id', 'movie_id', 'movie_title', 'rating', 'release_year', 'release_month', 
         'rating_year', 'rating_month', 'age_group', 'gender', 'occupation', 'region',  
         'unknown', 'Action','Adventure', 'Animation', "Children's", 'Comedy', 'Crime',
         'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
         'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', ]]

df_us

<ipython-input-23-51ba228bbedb>:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movielense_merged.csv')


,user_id,movie_id,movie_title,rating,release_year,release_month,rating_year,rating_month,age_group,gender,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,308,1,Toy Story (1995),4,1995,1,1998,2,6,M,...,0,0,0,0,0,0,0,0,0,0
1,308,4,Get Shorty (1995),5,1995,1,1998,2,6,M,...,0,0,0,0,0,0,0,0,0,0
2,308,5,Copycat (1995),4,1995,1,1998,2,6,M,...,0,0,0,0,0,0,0,1,0,0
3,308,7,Twelve Monkeys (1995),4,1995,1,1998,2,6,M,...,0,0,0,0,0,0,1,0,0,0
4,308,8,Babe (1995),5,1995,1,1998,2,6,M,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99986,729,748,"Saint, The (1997)",4,1997,3,1998,4,1,M,...,0,0,0,0,0,1,0,1,0,0
99987,729,751,Tomorrow Never Dies (1997),3,1997,1,1998,4,1,M,...,0,0,0,0,0,1,0,1,0,0
99988,729,879,"Peacemaker, The (1997)",3,1997,1,1998,4,1,M,...,0,0,0,0,0,0,0,1,1,0
99989,729,894,Home Alone 3 (1997),1,1997,1,1998,4,1,M,...,0,0,0,0,0,0,0,0,0,0


### CF 유사도 모델
참고
- https://www.youtube.com/watch?v=mJ8pEjN3FKg </br>
- https://www.youtube.com/watch?v=Lc5mfCF0mCU

In [ ]:
columns=['user_id', 'movie_id', 'rating', 'timestamp'] 
data=pd.read_csv('ml-100k/u.data', sep='\t', names=columns)
data = data[['user_id', 'movie_id', 'rating']]
data

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [ ]:
# 협업필터링을 이용한 영화 추천 엔진

n_users = data['user_id'].unique().shape[0] #943
n_items = data['movie_id'].nunique() #1682

#(943,1682)형태의 0으로 이루어진 행렬
data_matrix = np.zeros((n_users, n_items)) 

# sparse matrix 형태로 구성, 위 zeros행렬에 각 값 대치
for row in data.itertuples():
  data_matrix[row[1]-1, row[2]-1]=row[3]
# itertuple이용 행에 접근할 수 있는 iterator 생성
# -1은 인덱스와 다르기 때문
# row[1]-1: 사용자정보, row[2]-1: 영화정보, row[3]:평점

# 훈련 데이터와 텍스트 데이터의 분리
from sklearn.model_selection import train_test_split

train, test = train_test_split(data_matrix, test_size=0.2, random_state=42)

# 사용자 간 유사도 행렬 생성
from sklearn.metrics.pairwise import cosine_distances

cosine_distances(train)

# 코사인 유사도: 내적 공간의 두 벡터의 사이각에 대한 코사인을 측정해 유사도를 나타내는 척도
# cos(0) == 1, cos(θ) < 1
# cosine distance함수로 벡터만 주면 바로 계산 가능
distances = 1 - cosine_distances(train) # 자기자신을 1로 하도록 변경

# 평가 예측 및 모델 성능 측정
user_pred = distances.dot(train) / np.array([np.abs(distances).sum(axis=1)]).T
# np.dot(). 행의 크기가 같아야 연산 가능
# 

from sklearn.metrics import mean_squared_error
def get_mse(pred, actual):
  pred = pred[actual.nonzero()].flatten()
  actual = actual[actual.nonzero()].flatten()
  return mean_squared_error(pred, actual)

# 훈련데이터 RMSE
a = np.sqrt(get_mse(user_pred, train)) 
# 테스트 RMSE
b = np.sqrt(get_mse(user_pred, test)) 

print(a,b)

2.799242657919795 2.989599866496873


In [ ]:
data_matrix

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [24]:
df2 = df_us[['user_id', 'movie_id', 'rating']]
df2

,user_id,movie_id,rating
0,308,1,4
1,308,4,5
2,308,5,4
3,308,7,4
4,308,8,5
...,...,...,...
99986,729,748,4
99987,729,751,3
99988,729,879,3
99989,729,894,1


In [29]:
n_users = df2['user_id'].unique().shape[0] #943
n_items = df2['movie_id'].nunique() #1681

#(943,1681)형태의 0으로 이루어진 행렬
data_matrix2 = np.zeros((n_users, n_items))

In [30]:
data_matrix2.shape

(943, 1681)

In [31]:
# 협업필터링을 이용한 영화 추천 엔진

n_users = df2['user_id'].unique().shape[0] #943
n_items = df2['movie_id'].nunique() #1681

#(943,1681)형태의 0으로 이루어진 행렬
data_matrix2 = np.zeros((n_users, n_items)) 

# sparse matrix 형태로 구성, 위 zeros행렬에 각 값 대치
for row in df2.itertuples():
  data_matrix2[row[1]-1, row[2]-1]=row[3]
# itertuple이용 행에 접근할 수 있는 iterator 생성
# -1은 인덱스와 다르기 때문
# row[1]-1: 사용자정보, row[2]-1: 영화정보, row[3]:평점

# 훈련 데이터와 텍스트 데이터의 분리
from sklearn.model_selection import train_test_split

train2, test2 = train_test_split(data_matrix2, test_size=0.2, random_state=42)

# 사용자 간 유사도 행렬 생성
from sklearn.metrics.pairwise import cosine_distances

cosine_distances(train2)

# 코사인 유사도: 내적 공간의 두 벡터의 사이각에 대한 코사인을 측정해 유사도를 나타내는 척도
# cos(0) == 1, cos(θ) < 1
# cosine distance함수로 벡터만 주면 바로 계산 가능
distances2 = 1 - cosine_distances(train2) # 자기자신을 1로 하도록 변경

# 평가 예측 및 모델 성능 측정
user_pred2 = distances2.dot(train2) / np.array([np.abs(distances2).sum(axis=1)]).T
# np.dot(). 행의 크기가 같아야 연산 가능
# 

from sklearn.metrics import mean_squared_error
def get_mse(pred, actual):
  pred = pred[actual.nonzero()].flatten()
  actual = actual[actual.nonzero()].flatten()
  return mean_squared_error(pred, actual)

# 훈련데이터 RMSE
c = np.sqrt(get_mse(user_pred2, train2)) 
# 테스트 RMSE
d = np.sqrt(get_mse(user_pred2, test2)) 

print(c,d)

IndexError: ignored